# MVP Pipeline de Dados – Recursos Humanos  
## Análise de Retenção e Eficiência Organizacional  

**Vanessa Araújo**  

PUC-RJ – MBA em Ciência de Dados e Analytics  
Disciplina de Engenharia de Dados  

---

**Script ETL para carga na camada GOLD**

Carga de Dados
Os dados serão carregados a partir da camada silver gerando as métricas a serem persistidas na camada gold.

**1. gold.turnover_metrics — Rotatividade**

**Objetivo:** Analisar funcionários ativos vs desligados e calcular taxa de rotatividade por departamento.

**Estrutura da tabela:**

department

total_funcionarios

total_desligados

taxa_rotatividade

In [0]:
%sql
CREATE OR REPLACE TABLE gold.turnover_metrics AS
SELECT
    d.department_name AS department,
    COUNT(DISTINCT f.employee_id) AS total_funcionarios,
    
    SUM(
        CASE 
            WHEN f.termination_date IS NOT NULL THEN 1 
            ELSE 0 
        END
    ) AS total_desligados,
    
    ROUND(
        SUM(
            CASE 
                WHEN f.termination_date IS NOT NULL THEN 1 
                ELSE 0 
            END
        ) / COUNT(DISTINCT f.employee_id),
        4
    ) AS taxa_rotatividade,
    





---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-6774941890420307>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE OR REPLACE TABLE gold.turnover_metrics AS\nSELECT\n    d.department_name AS department,\n    COUNT(DISTINCT f.employee_id) AS total_funcionarios,\n    \n    SUM(\n        CASE \n            WHEN f.termination_date IS NOT NULL THEN 1 \n            ELSE 0 \n        END\n    ) AS total_desligados,\n    \n    ROUND(\n        SUM(\n            CASE \n                WHEN f.termination_date IS NOT NULL THEN 1 \n                ELSE 0 \n            END\n        ) / COUNT(DISTINCT f.employee_id),\n        4\n    ) AS taxa_rotatividade,\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magi

In [0]:
%sql
SELECT *
FROM gold.turnover_metrics
LIMIT 20;

department,total_funcionarios,total_desligados,taxa_rotatividade,tempo_medio_empresa
Escritório Executivo,1,0,0.0,4919.0
Venda,32,15,0.4688,4046.41
Engenharia de Software,10,9,0.9,3626.1
Produção,208,249,1.1971,3309.1
Escritórios administrativos,10,9,0.9,3482.9
T.I,50,30,0.6,3380.66


**2. gold.absenteeism_metrics — Absenteísmo**

**Objetivo**: Avaliar faltas e atrasos por departamento.

**Estrutura:**

department

total_ausencias

media_ausencias

total_atrasos

In [0]:
%sql
CREATE OR REPLACE TABLE gold.absenteeism_metrics AS
SELECT
    d.department_name AS department,
    COUNT(DISTINCT f.employee_id) AS total_funcionarios,
    SUM(f.absences) AS total_ausencias,
    ROUND(
        AVG(f.absences),
        2
    ) AS media_ausencias_por_funcionario,
    SUM(f.days_late_last_30) AS total_atrasos_ultimos_30_dias
FROM silver.fact_employee f
JOIN silver.dim_department d
    ON f.department_id = d.department_id
GROUP BY d.department_name;



num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.absenteeism_metrics
LIMIT 10;

department,total_funcionarios,total_ausencias,media_ausencias_por_funcionario,total_atrasos_ultimos_30_dias
Escritório Executivo,1,50,10.0,0
Venda,32,1800,11.25,110
Engenharia de Software,10,460,9.2,20
Produção,208,10590,10.18,480
Escritórios administrativos,10,410,8.2,0
T.I,50,2610,10.44,35


**3.gold.performance_metrics — Desempenho**

**Objetivo:**Analisar a distribuição de desempenho e sua relação com ausências.

**Estrutura:**

department

performance_rating

qtd_funcionarios

media_ausencias

In [0]:
%sql
CREATE OR REPLACE TABLE gold.performance_metrics AS
SELECT
    d.department_name AS department,
    f.performance_score,
    COUNT(DISTINCT f.employee_id) AS total_funcionarios,
    SUM(f.absences) AS total_ausencias,
    SUM(f.days_late_last_30) AS total_atrasos_ultimos_30_dias
FROM silver.fact_employee f
JOIN silver.dim_department d
    ON f.department_id = d.department_id
GROUP BY
    d.department_name,
    f.performance_score;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.performance_metrics
LIMIT 20;

department,performance_score,total_funcionarios,total_ausencias,total_atrasos_ultimos_30_dias
Engenharia de Software,Atende totalmente,7,295,0
T.I,PMD,1,10,15
Venda,Precisa de melhorias,1,80,10
Engenharia de Software,Precisa de melhorias,1,95,20
Produção,Precisa de melhorias,15,780,290
Venda,Excede,2,100,0
T.I,Excede,6,370,0
Engenharia de Software,Excede,2,70,0
Escritório Executivo,Atende totalmente,1,50,0
Venda,PMD,4,170,75


**4. gold.satisfaction_metrics — Satisfação**

**Objetivo:**Comparar satisfação média por departamento e status.

**Estrutura:**

department

status_funcionario

media_satisfacao

In [0]:
%sql
CREATE OR REPLACE TABLE gold.satisfaction_metrics AS
SELECT
    d.department_name AS department,
    CASE 
        WHEN f.termination_date IS NULL THEN 'Ativo'
        ELSE 'Desligado'
    END AS status_funcionario,
    COUNT(DISTINCT f.employee_id) AS total_funcionarios,
    ROUND(AVG(1 + (f.employee_satisfaction - 43) * 4.0 / 457), 2) AS media_satisfacao
FROM silver.fact_employee f
JOIN silver.dim_department d
    ON f.department_id = d.department_id
GROUP BY
    d.department_name,
    CASE 
        WHEN f.termination_date IS NULL THEN 'Ativo'
        ELSE 'Desligado'
    END;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.satisfaction_metrics
LIMIT 20;

department,status_funcionario,total_funcionarios,media_satisfacao
T.I,Desligado,10,4.47
Venda,Ativo,27,3.89
Engenharia de Software,Desligado,3,4.04
Engenharia de Software,Ativo,7,4.23
Escritórios administrativos,Ativo,7,4.69
T.I,Ativo,40,4.21
Escritório Executivo,Ativo,1,4.85
Produção,Desligado,83,4.21
Produção,Ativo,125,4.25
Escritórios administrativos,Desligado,3,3.89


**5. gold.diversity_metrics — Diversidade**

**Objetivo:**Avaliar distribuição racial por departamento.

**Estrutura:**

department

race

qtd_funcionarios

In [0]:
%sql
CREATE OR REPLACE TABLE gold.diversity_metrics AS
SELECT
    d.department_name             AS department,
    e.race                        AS race,
    e.gender                      AS gender,
    e.hispanic_latino             AS latino,
    COUNT(DISTINCT f.employee_id) AS total_funcionarios
FROM silver.fact_employee f
JOIN silver.dim_employee e
    ON f.employee_id = e.employee_id
JOIN silver.dim_department d
    ON f.department_id = d.department_id
GROUP BY
    d.department_name,
    e.race,
    e.gender,
    e.hispanic_latino;



num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.diversity_metrics

department,race,gender,latino,total_funcionarios
Venda,Negro ou afro-americano,H,Não,4
Venda,Índio americano ou nativo do Alasca,H,Não,1
Venda,Branco,H,Não,7
T.I,Negro ou afro-americano,H,Sim,1
Engenharia de Software,Negro ou afro-americano,M,Não,2
Produção,Negro ou afro-americano,H,Sim,1
Venda,Negro ou afro-americano,H,Sim,1
Engenharia de Software,Negro ou afro-americano,H,Não,2
Produção,Branco,H,Sim,7
Escritórios administrativos,Branco,M,Não,2


**6. gold.salary_metrics — Massa Salarial**

**Objetivo**: Analisar massa salarial e média salarial por departamento e cargo.

**Estrutura:**

department

job_title

massa_salarial

salario_medio

In [0]:
%sql
CREATE OR REPLACE TABLE gold.salary_metrics AS
SELECT
    d.department_name                              AS department,
    COUNT(DISTINCT f.employee_id)                  AS total_funcionarios,
    ROUND(SUM(f.salary), 2)                        AS massa_salarial_total,
    ROUND(AVG(f.salary), 2)                        AS salario_medio,
    ROUND(MIN(f.salary), 2)                        AS salario_minimo,
    ROUND(MAX(f.salary), 2)                        AS salario_maximo
FROM silver.fact_employee f
JOIN silver.dim_department d
    ON f.department_id = d.department_id
GROUP BY d.department_name;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM gold.salary_metrics
ORDER BY massa_salarial_total DESC;

department,total_funcionarios,massa_salarial_total,salario_medio,salario_minimo,salario_maximo
Produção,208,6.23665E7,59967.79,45046.0,170500.0
T.I,50,2.426616E7,97064.64,50178.0,220450.0
Venda,32,1.098945E7,68684.06,55875.0,180000.0
Engenharia de Software,10,4807605.0,96152.1,77692.0,108987.0
Escritórios administrativos,10,3647450.0,72949.0,49920.0,106367.0
Escritório Executivo,1,1250000.0,250000.0,250000.0,250000.0
